**Data Loading**

Import libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import warnings

Unzip master data

In [ ]:
master_data_path=''
!unzip /content/master_data.zip

Load data to pandas df

In [ ]:
# read data
data_path="/content/master_data.csv"
houston_data = pd.read_csv(data_path)
houston_data.head()

Get general information and null values count

In [ ]:
# General Info
no_columns=len(houston_data.columns)
print('Total columns',":",no_columns)
print('Total data count',len(houston_data))
# Find null values for each column
print("Null values info")
nulls=houston_data.isnull().sum(axis = 0)
pos=0
for i in nulls:
  print(pos,houston_data.columns[pos],":",i,'Type :',houston_data.dtypes[pos])
  pos+=1

**Data Distribution Analysis**

In [ ]:
# import matplotlib.pyplot as plt
houston_data.boxplot(column=['Vacancy %'])

In [ ]:
# features = houston_data.drop('', axis=1)
labels1 = houston_data['Vacancy %']
labels2 = houston_data['Vacant SF Direct']

In [ ]:
sampled_labels=labels1.sample(n=100000,random_state=1)
sampled_labels.plot(kind='hist')

In [ ]:
houston_data_sub=houston_data[['Property ID','Vacancy %']]
Data_based_Id=houston_data_sub.groupby(['Property ID']).mean()

In [ ]:
print('Before grouping',len(houston_data_sub))
print('After grouping',len(Data_based_Id))

In [ ]:
Data_based_Id.boxplot(column=['Vacancy %'])

In [ ]:
# Data_based_Id.head()
sampled_labels=Data_based_Id.sample(n=10000,random_state=1)
sampled_labels.plot(kind='hist')

**Data Cleaning and Manipulation**

Set an updated dataframe that converts categorical values to numerical OHE

In [ ]:
import numpy as np

def convert_name(original_name,name):
  # print(name)
  # if name == "nan":
  #   return original_name+": na"
  return original_name+":"+name

# del houston_data_updated
dummy_secondary_type=pd.get_dummies(houston_data['Secondary Type'])
# dummy_secondary_type.rename(columns={np.nan: "Secondary Type Na"})
dummy_secondary_type=dummy_secondary_type.rename(columns={x: convert_name('Secondary Type',x) for x in dummy_secondary_type.columns})
# dummy_secondary_type.head()
houston_data_updated=pd.concat([houston_data, dummy_secondary_type], axis=1)
houston_data_updated = houston_data_updated.drop(['Secondary Type'], axis=1)
del dummy_secondary_type
# houston_data_updated.head()


Set columns for dropping and convertion

In [ ]:
# columns to drop
columns_drop = []
# columns for OHE
columns_ohe = []

In [ ]:
# columns to convert from Y,N to 1,0
columns_01 = []

Drop columns

In [ ]:
houston_data_updated = houston_data_updated.drop(columns_drop, axis=1)

Convert OHE using loop

In [ ]:
for col_name in columns_ohe:
  
  if col_name in houston_data_updated.columns:
    dummy_type=pd.get_dummies(houston_data[col_name])
    dummy_type=dummy_type.rename(columns={x: convert_name(col_name,x) for x in dummy_type.columns})
    houston_data_updated=pd.concat([houston_data_updated, dummy_type], axis=1)
    del dummy_type
    houston_data_updated = houston_data_updated.drop([col_name], axis=1)
houston_data_updated.head()

Covert to 0,1

In [ ]:
for col_name in columns_01:
  houston_data_updated[col_name]=houston_data_updated[col_name].replace('N',0)
  houston_data_updated[col_name]=houston_data_updated[col_name].replace('Y',1)
houston_data_updated.head()

Test for columns with object type

In [ ]:
# find all object columns
k=0
for i in houston_data_updated.columns:
  if houston_data_updated.dtypes[k]=="object":
    print(i)
  k+=1

Split data

In [ ]:
columns_drop_corr=[]

houston_data_labels = houston_data_updated['label name']
houston_data_features = houston_data_updated.drop(columns_drop_corr, axis=1)


X_train, X_test, y_train, y_test = train_test_split(houston_data_features, houston_data_labels, test_size=0.4, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

Scale Data

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
scaler.fit(X_train)


StandardScaler(copy=True, with_mean=False, with_std=True)

In [ ]:
x_train_scaled=scaler.transform(X_train)
# x_val_scaled=scaler.transform(X_val)
x_test_scaled=scaler.transform(X_test)


**Machine Learning - XGBoost**

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

XGBoost Parameters: https://xgboost.readthedocs.io/en/latest/parameter.html

GPU seetings:https://xgboost.readthedocs.io/en/latest/gpu/index.html

In [ ]:
# Print function
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))
    print('BEST SCORE',results.best_score_)
    means = results.cv_results_['mean_test_score']
    scores = results.scorer_
    print(scores)
    for score, params in zip(means, results.cv_results_['params']):
        print('{} for {}'.format(round(score, 3), params))

Scoring: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
# User defined MSE
from sklearn.metrics import r2_score, mean_squared_error, make_scorer

def MSE(y_true,y_pred):
    mse = mean_squared_error(y_true, y_pred)
    GPUtil.showUtilization()
    print('MSE: %2.3f' % mse)
    return mse

def two_score(y_true,y_pred):    
    score = MSE(y_true,y_pred) 
    return score

def two_scorer():
    return make_scorer(two_score, greater_is_better=False) 

**Define and Train Model**

Gridsearch: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
xgb_param_grid = {'learning_rate': [0.1,0.5,1],
                   'max_depth': [15,20,25],
                  'min_child_weight': [0.5,1,1.5,2],
                  'gamma': [0.1,0.2]                  
                  }

model = XGBRegressor(objective='reg:squarederror',tree_method='gpu_hist',gpu_id=0,n_jobs=16)
grid = GridSearchCV(model, xgb_param_grid,scoring=two_scorer())

# Train with unscaled data
grid.fit(X_train,y_train)

# model.fit(X_train, y_train)
joblib.dump(grid.best_estimator_, model_drive_path)
print_results(grid)

| ID | GPU  | MEM |
-------------------
|  0 | 100% | 16% |
MSE: 0.014
| ID | GPU  | MEM |
-------------------
|  0 | 100% | 16% |
MSE: 0.014
| ID | GPU | MEM |
------------------
|  0 | 98% | 16% |
MSE: 0.014
| ID | GPU | MEM |
------------------
|  0 | 57% | 16% |
MSE: 0.014
| ID | GPU  | MEM |
-------------------
|  0 | 100% | 16% |
MSE: 0.014
| ID | GPU  | MEM |
-------------------
|  0 | 100% | 16% |
MSE: 0.014
| ID | GPU | MEM |
------------------
|  0 | 96% | 16% |
MSE: 0.014
| ID | GPU  | MEM |
-------------------
|  0 | 100% | 16% |
MSE: 0.014
| ID | GPU | MEM |
------------------
|  0 | 92% | 16% |
MSE: 0.014
| ID | GPU  | MEM |
-------------------
|  0 | 100% | 16% |
MSE: 0.014
| ID | GPU | MEM |
------------------
|  0 | 60% | 16% |
MSE: 0.014
| ID | GPU | MEM |
------------------
|  0 | 83% | 16% |
MSE: 0.014
| ID | GPU  | MEM |
-------------------
|  0 | 100% | 16% |
MSE: 0.014
| ID | GPU  | MEM |
-------------------
|  0 | 100% | 16% |
MSE: 0.014
| ID | GPU | MEM |
-----

**Test Model**

In [ ]:
y_test_pred=grid.predict(X_test)

RMS Test

In [ ]:
test_rms=mean_squared_error(y_test_pred,y_test)
print('Test MSE Error :',test_rms)

Sample data for prediction and target

In [ ]:
i=0
for x,y in zip(y_test_pred,y_test):
  print('prediction:',x,'real',y)
  if i==100:
    break
  i+=1


**Feature Importance**

In [ ]:
pos=0
feat_imp = grid.best_estimator_.feature_importances_
feature_list=[]
for i in feat_imp:
  feature_list.append((houston_data_features.columns[pos],i))
  pos+=1
feature_list_sorted=sorted(feature_list, key=lambda x: x[1], reverse=True)
# print(feature_list_sorted)
pos=0
for i in feature_list_sorted:
  print(pos,":",i)
  pos+=1

In [ ]:
import csv

def move_to_csv(file,x,y):
    with open(file, mode='a',newline='') as data_file:
        data_writer = csv.writer(data_file, delimiter=',')
        data_writer.writerow([x,y])

In [ ]:
# store feature list in csv
file_out='Feature_Imp.csv'

for i in feature_list_sorted:
  move_to_csv(file_out,i[0],i[1])

**Save the model in pkl file**

In [ ]:
# connect to drive to save model
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_path='../../../XGB_non_scaled.pkl'
grid_path = '../../../Grid_non_scaled.pkl'
drive_path='/content/drive/My Drive/4CModels/'
model_name='XGB_non_scaled_1.pkl'
model_drive_path=drive_path+model_name

In [ ]:
joblib.dump(grid.best_estimator_, model_drive_path)

**Load Model**

In [ ]:
model_loaded = XGBRegressor(objective='reg:squarederror',tree_method='gpu_hist',gpu_id=0,n_jobs=16)
# model_loaded.load_model(model_path)  # load data
model_loaded = joblib.load(model_drive_path)

In [ ]:
pos=0
feat_imp = model_loaded.feature_importances_
feature_list=[]
for i in feat_imp:
  feature_list.append((houston_data_features.columns[pos],i))
  pos+=1
feature_list_sorted=sorted(feature_list, key=lambda x: x[1], reverse=True)
# print(feature_list_sorted)
pos=0
for i in feature_list_sorted:
  print(pos,":",i)
  pos+=1

In [ ]:
y_pred=model_loaded.predict(X_test)

In [ ]:
# store prediction in csv
file_out='xgboost_prediction.csv'

for x,y in zip(y_pred,y_test):
  move_to_csv(file_out,x,y)

**Unofficial code: Additional Code for debug and trial and error**

Dummy code to check column type and category size

In [ ]:
col_name=''
dummy_type=pd.get_dummies(houston_data[col_name])
dummy_type=dummy_type.rename(columns={x: convert_name(col_name,x) for x in dummy_type.columns})
dummy_type


Use wandb to track training (not needed)

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.8MB 2.7MB/s 
     |████████████████████████████████| 133kB 17.7MB/s 
     |████████████████████████████████| 102kB 8.0MB/s 
     |████████████████████████████████| 163kB 18.2MB/s 
     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=8746d27a699df7f8b4db9262e6331138a354a40964af32d22e3d8365d768cac2
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=1fbf290dfb626fee972a64714a66ddf83a04846c549d0147e53ec47af4441196
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=95ef4ddf789ec96022b0e3be1a0bac50ce18e1b04d3822

In [ ]:
import wandb
wandb.init()

Load data (not used)

In [ ]:
!pip install gputil
!pip install psutil
!pip install humanize

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=d91f1fb445caafbfb4f4318f0bd86778aabd2dadd1f6bc43fd674c5f83e0d540
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [ ]:
import GPUtil
c=GPUtil.showUtilization()
# GPUtil.getGPUs
GPUtil.getAvailable()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


[0]